In [1]:
import requests
from web3 import Web3
import pandas as pd
import sys
sys.path.append("./chainklik")

import libs.common.utils as utils
import libs.aave.api as aave_api
import libs.common.payload as payload
import config.config as cfg

In [2]:
# Connect to an Ethereum node
w3 = Web3(Web3.HTTPProvider(cfg.config["eth_sepolia_http_url"]))

# Set sender and recipient addresses
sender_address = '0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7'
recipient_address = '0xaf6667a2F847beeca6a6604126Dc28344518840b'

# Set private key for the sender's account. 
private_key = '1dab201501e8b882ca3413edcdfed263e4834cd8ea4c9586aca7fb699c51d681'

In [9]:
balance_sender = w3.from_wei(w3.eth.get_balance(sender_address), 'ether')
balance_recipient = w3.from_wei(w3.eth.get_balance(recipient_address), 'ether')

print(f'The balance of { sender_address } is: { balance_sender } ETH')
print(f'The balance of { recipient_address } is: { balance_recipient } ETH')

The balance of 0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7 is: 0.489531614690461 ETH
The balance of 0xaf6667a2F847beeca6a6604126Dc28344518840b is: 0.01 ETH


In [8]:
transaction = w3.eth.account.sign_transaction(transaction_params, private_key)

# Send the transaction
transaction_hash = w3.eth.send_raw_transaction(transaction.rawTransaction)

# Wait for the transaction to be mined
transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)

# Check the transaction status
if transaction_receipt.status:
    print('Transaction successful!')
    print('Transaction hash:', transaction_hash.hex())
    print(f'Explorer link: https://sepolia.etherscan.io/tx/{transaction_hash.hex()}')
else:
    print('Transaction failed.')

# Fetch balance data after the transaction
balance_sender = w3.from_wei(w3.eth.get_balance(sender_address), 'ether')
balance_recipient = w3.from_wei(w3.eth.get_balance(recipient_address), 'ether')

print(f'The balance of { sender_address } is: { balance_sender } ETH')
print(f'The balance of { recipient_address } is: { balance_recipient } ETH')

Transaction successful!
Transaction hash: 0xa86796e7a8d2654a587ea9b1fa40b82265b0b5bef4100de68b09938fc7bd72ec
Explorer link: https://sepolia.etherscan.io/tx/0xa86796e7a8d2654a587ea9b1fa40b82265b0b5bef4100de68b09938fc7bd72ec
The balance of 0x5bA4D4264Bf9A8C3aaF7e1fea6f83f50643A3Fd7 is: 0.489531614690461 ETH
The balance of 0xaf6667a2F847beeca6a6604126Dc28344518840b is: 0.01 ETH


In [ ]:
class txNode:
    def __init__(self, wallet, to, amount, finality = 2, gas = 21000):
        self.wallet = wallet
        self.to = to
        self.amount = amount
        self.finality = finality
        self.gas = gas
        self.status = "new"
    
    def run(ctx):
        if self.status == "new":
            latest_block = w3.eth.get_block("latest")
            base_fee_per_gas = latest_block.baseFeePerGas   # Base fee in the latest block (in wei)
            max_priority_fee_per_gas = w3.to_wei(1, 'gwei') # Priority fee to include the transaction in the block
            max_fee_per_gas = (5 * base_fee_per_gas) + max_priority_fee_per_gas # Maximum amount you’re willing to pay 
            
            transaction_params = {
                'from': wallet,
                'to': to,
                'value': w3.to_wei(amount, 'ether'),
                'nonce': w3.eth.get_transaction_count(wallet),
                'gas': gas, 
                'maxFeePerGas': max_fee_per_gas, # Maximum amount you’re willing to pay 
                'maxPriorityFeePerGas': max_priority_fee_per_gas, # Priority fee to include the transaction in the block
                'chainId': 11155111 # ChainId of Sepolia Testnet
            }
        
            transaction = w3.eth.account.sign_transaction(transaction_params, private_key)
            transaction_hash = w3.eth.send_raw_transaction(transaction.rawTransaction)
            transaction_receipt = w3.eth.wait_for_transaction_receipt(transaction_hash)
        
            if transaction_receipt.status:
                print('Transaction successful!')
                print('Transaction hash:', transaction_hash.hex())
                print(f'Explorer link: https://sepolia.etherscan.io/tx/{transaction_hash.hex()}')
                self.txBlock = transaction_receipt["blockNumber"]
                self.status = "unfinalized"
            else:
                print('Transaction failed.')
                self.status = "failed"
        elif self.status == "unfinalized" and ctx.blockNumber - self.txBlock >= self.finality:
            self.status = "finalized"
            self.active = True
        elif self.status == "finalized":
            self.active = False

In [ ]:
class addNode:
    def __init__(self, in1, in2):
        self.in1 = in1
        self.in2 = in2

    

In [12]:
# transactions
# function calls - read
# function calls - write

5006376

In [13]:
# 1. transaction path
# send one transaction and wait for finality
# trigger second transaction
# timer -> txn 1 -> finality -> txn 2

In [14]:
# 2. aave collateral auto management
# listen to healthfactor on a regular interval
# when healthfactor drops below a level, post collateral
# timer -> healthfactor (interval, every block) -> condition filter -> txn (deposit)

In [15]:
# 3. curve trades price dependent
# monitor curve pool price
# set a level to trigger the trade
# make a swap when level is reached
# timer -> curve pool price (interval, every block) -> condition filter -> txn (swap)

In [16]:
# 4. cross chain transaction path
# send one txn on eth
# wait for it to finalize
# send second txn on polygon
# timer -> txn 1 -> finality -> txn 2

In [220]:
from abc import ABC, abstractmethod

class Timer(ABC):
    def __init__(self, start, end, frequency):
        self.start = start
        self.end = end
        self.frequency = frequency
        assert self.start < self.end

    @abstractmethod
    def is_active(self, ctx):
        pass

    @abstractmethod
    def is_expired(self, ctx):
        pass
        
class BlockTimer(Timer):
    def __init__(self, start, end, frequency):
        super().__init__(start, end, frequency)

    def is_active(self, ctx):
        if ctx["block_time"] - self.start >= 0:
            self.start = ((ctx["block_time"] - self.start) // self.frequency + 1) * self.frequency + self.start
            return True
        else:
            return False

    def is_expired(self, ctx):
        if self.start >= self.end:
            return True
        return False

In [221]:
class TimerDecorator:
    def __init__(self, timer_attr):
        self.timer_attr = timer_attr

    def __call__(self, func):
        def wrapper(*args, **kwargs):
            timer = getattr(args[0], self.timer_attr)
            ctx = args[1]
            if not timer.is_expired(ctx) and timer.is_active(ctx):
                func(*args, **kwargs)
        return wrapper

In [222]:
class Node(ABC):
    def __init__(self, deps: dict, params: dict):
        self.deps = deps
        self.params = params
        self.output = None

    @abstractmethod
    def run(self, ctx: dict, values: dict) -> None:
        pass

In [223]:
class DataNode(Node):
    def __init__(self, deps: dict, params: dict, timer: Timer = None):
        super().__init__(deps, params)
        self.timer = timer
    
    @TimerDecorator("timer")
    def run(self, ctx: dict, values: dict) -> None:
        self.output = values

In [224]:
bt = BlockTimer(0, 100, 15)
node = DataNode({}, {}, bt)
for t in [5,7,11,15,35,62,70,80,95,100,105,120]:
    ctx = {
        "block_time" : t
    }

    values = {
        "value" : t
    }

    node.run(ctx, values)
    print(t, node.output, bt.__dict__)

5 {'value': 5} {'start': 15, 'end': 100, 'frequency': 15}
7 {'value': 5} {'start': 15, 'end': 100, 'frequency': 15}
11 {'value': 5} {'start': 15, 'end': 100, 'frequency': 15}
15 {'value': 15} {'start': 30, 'end': 100, 'frequency': 15}
35 {'value': 35} {'start': 45, 'end': 100, 'frequency': 15}
62 {'value': 62} {'start': 75, 'end': 100, 'frequency': 15}
70 {'value': 62} {'start': 75, 'end': 100, 'frequency': 15}
80 {'value': 80} {'start': 90, 'end': 100, 'frequency': 15}
95 {'value': 95} {'start': 105, 'end': 100, 'frequency': 15}
100 {'value': 95} {'start': 105, 'end': 100, 'frequency': 15}
105 {'value': 95} {'start': 105, 'end': 100, 'frequency': 15}
120 {'value': 95} {'start': 105, 'end': 100, 'frequency': 15}


In [225]:
bt = BlockTimer(0, 10, 10)
node = DataNode({}, {}, bt)
for t in [1,5,7,11,15,35,62,70,80,95,100,105,120]:
    ctx = {
        "block_time" : t
    }

    values = {
        "value" : t
    }

    node.run(ctx, values)
    print(node.output, bt.__dict__)

{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
{'value': 1} {'start': 10, 'end': 10, 'frequency': 10}
